In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier

In [69]:
# loading the data in 

file_path = Path("Resources/teamSummary.csv")

df = pd.read_csv(file_path)

df

,posteam,Total Offensive Plays,Team Pass(%),Team Rush(%),Pass Attempts,Pass Completions,Pass Completion(%),Pass Yds,Yds Per Pass Attempt,Yds Per Pass Completion,Pass TDs,Pass TDs (%GT),Rush Attempts,Rush Yards,Yds per Rush,Rush TDs,Rush TDs (%GT)
0,ARI,1166,57.38%,42.62%,669,434,64.87%,4756.0,7.109118,10.958525,24,50.94%,497,2157.0,4.340040,24,45.28%
1,ATL,998,61.52%,38.48%,614,377,61.40%,3987.0,6.493485,10.575597,11,58.82%,384,1461.0,3.804688,11,32.35%
2,BAL,1177,56.75%,43.25%,668,396,59.28%,4267.0,6.387725,10.775253,18,48.84%,509,2487.0,4.886051,18,41.86%
3,BUF,1240,60.56%,39.44%,751,463,61.65%,5087.0,6.773635,10.987041,23,66.18%,489,2514.0,5.141104,23,33.82%
4,CAR,1099,59.42%,40.58%,653,348,53.29%,3573.0,5.471669,10.267241,17,42.42%,446,1855.0,4.159193,17,51.52%
5,CHI,1067,56.33%,43.67%,601,332,55.24%,3635.0,6.048253,10.948795,14,50.00%,466,2028.0,4.351931,14,43.75%
6,CIN,1286,60.03%,39.97%,772,482,62.44%,5917.0,7.664508,12.275934,17,67.74%,514,2099.0,4.083658,17,27.42%
7,CLE,1043,54.55%,45.45%,569,320,56.24%,3619.0,6.360281,11.309375,20,50.00%,474,2483.0,5.238397,20,47.62%
8,DAL,1212,60.48%,39.52%,733,468,63.85%,5233.0,7.139154,11.181624,16,69.49%,479,2204.0,4.601253,16,27.12%
9,DEN,1032,56.49%,43.51%,583,354,60.72%,3856.0,6.614065,10.892655,16,51.28%,449,2031.0,4.523385,16,41.03%


In [75]:
X = df.dropna(axis = 'columns' , how='any')

X = df.dropna()


In [86]:
X = df.drop('Team Rush(%)' , axis=1)

X = pd.get_dummies(X)

# Creating Target 

y = df['Team Rush(%)']

X.describe()

,Total Offensive Plays,Pass Attempts,Pass Completions,Pass Yds,Yds Per Pass Attempt,Yds Per Pass Completion,Pass TDs,Rush Attempts,Rush Yards,Yds per Rush,...,Rush TDs (%GT)_43.75%,Rush TDs (%GT)_44.00%,Rush TDs (%GT)_44.44%,Rush TDs (%GT)_45.28%,Rush TDs (%GT)_46.15%,Rush TDs (%GT)_46.43%,Rush TDs (%GT)_47.62%,Rush TDs (%GT)_50.00%,Rush TDs (%GT)_51.52%,Rush TDs (%GT)_54.17%
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.00000,32.00000,32.000000,32.000000,...,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,1117.687500,656.562500,398.156250,4369.906250,6.629199,10.964912,16.37500,461.12500,2051.750000,4.435731,...,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250,0.031250
std,95.641278,82.467564,65.503563,820.145399,0.641106,0.759468,5.17282,52.45259,352.266578,0.451871,...,0.176777,0.176777,0.176777,0.176777,0.176777,0.176777,0.176777,0.176777,0.176777,0.176777
min,947.000000,542.000000,293.000000,3437.000000,5.471669,9.321586,8.00000,376.00000,1426.000000,3.411483,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1042.750000,599.250000,353.250000,3728.000000,6.171372,10.562316,12.00000,417.50000,1824.750000,4.131307,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1098.000000,635.000000,387.000000,3976.000000,6.573095,11.030558,16.00000,457.50000,2029.500000,4.437384,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1173.250000,715.000000,445.750000,4982.500000,7.079075,11.271064,20.50000,494.00000,2301.500000,4.600617,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1321.000000,851.000000,551.000000,6121.000000,7.980738,12.880829,26.00000,573.00000,2822.000000,5.244353,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [87]:
# Checking the Balance of our target 

y.value_counts()

36.60%    2
42.62%    1
39.65%    1
48.21%    1
32.62%    1
47.91%    1
42.77%    1
35.68%    1
49.29%    1
36.29%    1
39.64%    1
47.79%    1
44.94%    1
41.26%    1
39.54%    1
36.71%    1
38.48%    1
37.70%    1
46.65%    1
41.55%    1
40.45%    1
40.34%    1
43.51%    1
39.52%    1
45.45%    1
39.97%    1
43.67%    1
40.58%    1
39.44%    1
43.25%    1
44.28%    1
Name: Team Rush(%), dtype: int64

In [88]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train.shape

(24, 168)

In [89]:

brfc = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [90]:
y_pred = brfc.predict(X_test)

balanced_accuracy_score(y_test , y_pred)

0.0

In [54]:
confusion_matrix(y_test, y_pred)
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

     35.68%       0.00      0.00      0.88      0.00      0.00      0.00       0.0
     36.60%       0.00      0.00      1.00      0.00      0.00      0.00       2.0
     39.44%       0.00      0.00      1.00      0.00      0.00      0.00       1.0
     39.52%       0.00      0.00      0.75      0.00      0.00      0.00       0.0
     39.64%       0.00      0.00      1.00      0.00      0.00      0.00       1.0
     42.77%       0.00      0.00      1.00      0.00      0.00      0.00       1.0
     43.25%       0.00      0.00      0.62      0.00      0.00      0.00       0.0
     44.94%       0.00      0.00      1.00      0.00      0.00      0.00       1.0
     45.45%       0.00      0.00      0.75      0.00      0.00      0.00       0.0
     47.79%       0.00      0.00      1.00      0.00      0.00      0.00       1.0
     47.91%       0.00      0.00      1.00      0.00      0.00      0.00       1.0

av